In [3]:
# Importing necessary modules
from tools import DataTools
# Importing necessary modules
from data import Data
# Importing necessary modules
from datetime import datetime
# Importing necessary modules
import pandas as pd
# Importing necessary modules
from sklearn.linear_model import LinearRegression
# Importing necessary modules
import matplotlib.pyplot as plt
# Importing necessary modules
import seaborn as sns
# Importing necessary modules
import gc
# Add import for the time module
import time

### 🕒 Étape 1 : Définir la date d'analyse
Dans cette étape, nous définissons une date et une heure spécifiques à analyser. Cette date servira à charger les fichiers de données correspondants à l’année choisie.

Notre code fonctionne de manière flexible, si l'on veut toutes les données d'une année il suffit de renseigner l'année souhaitée, si l'on veut le détail sur un mois, on lui donne le mois, et si l'on veut le détail que sur certains jours on peut aussi lui spécifier la date et l'heure d'un jour en particulier. 

In [5]:
specified_time = "2022-11-29 10:00"
# Convert string to datetime object
specified_time = datetime.strptime(specified_time, "%Y-%m-%d %H:%M")
# Format the datetime object to extract the year
formatted_time = specified_time.strftime("%Y")

# URL de l'API JCDecaux pour récupérer tous les contrats
contracts_url = "https://api.jcdecaux.com/vls/v3/contracts"
api_key = "412c37eac090528b3f24fe5843badacb6f3e907f"  # Remplacez par votre clé API

### 📂 Étape 2 : Charger les données des stations de vélos
Nous chargeons ici les fichiers contenant les métadonnées des stations de vélos, comme leur position, leur capacité et leur ville associée. Ces informations sont utilisées plus tard pour enrichir les données d'utilisation des vélos.

In [ ]:
path = r"..\data_files"
data_directory = r"..\data_files\data"
correct_header_data_bikes = ["city", "id", "request_date", "datetime", "bikes"]

# Open and read data files from the specified directory
station_data = DataTools.open_files_in_directory(path, "bike_station", "\t")
bike_station = Data()
# Load the opened data into a Data object
bike_station.get_data(station_data)
# Filter the data by city or station name
bike_station.filter_dataframes("city", ["amiens", "marseille"])
del station_data

### 🧪 Étape 3 : Charger les données de pollution et de météo
On importe les données mesurées par les stations de pollution et météo. Ces données sont cruciales pour les corrélations futures avec l’utilisation des vélos.

In [ ]:
# Open and read data files from the specified directory
station_data = DataTools.open_files_in_directory(path, "pollution_station", ",")
pollution_station = Data()
# Load the opened data into a Data object
pollution_station.get_data(station_data)
# Filter the data by city or station name
pollution_station.filter_dataframes("city", ["amiens", "marseille"])
del station_data

# Open and read data files from the specified directory
weather_data = DataTools.open_files_in_directory(data_directory, f"weather_{formatted_time}", ",")
data_weather = Data()
# Load the opened data into a Data object
data_weather.get_data(weather_data)
# Filter the data by city or station name
data_weather.filter_dataframes("name", ["Amiens", "Marseille"])
del weather_data

# Open and read data files from the specified directory
pollution_data = DataTools.open_files_in_directory(data_directory, f"pollution_{formatted_time}", ",")
data_pollution = Data()
# Load the opened data into a Data object
data_pollution.get_data(pollution_data)
# Filter the data by city or station name
data_pollution.filter_dataframes("name", ["Amiens"])
del pollution_data

### 🚲 Étape 4 : Charger et fusionner les données des vélos
Les données d’utilisation des vélos (nombre de vélos disponibles, etc.) sont chargées. Ensuite, on les enrichit en fusionnant avec les métadonnées des stations pour ajouter la géolocalisation, la capacité et l’identifiant de pollution.

In [ ]:
# Open and read data files from the specified directory
bike_data = DataTools.open_files_in_directory(data_directory, f"bike_{formatted_time}", "\t")
# Standardize column headers in the bike data
bike_data = DataTools.rename_header(bike_data, correct_header_data_bikes, keep_old_header=True)

data_bike = Data()
# Load the opened data into a Data object
data_bike.get_data(bike_data)
# Filter the data by city or station name
data_bike.filter_dataframes("city", ["amiens", "marseille"])
del bike_data

# Merge additional information (e.g., location, capacity) into bike or pollution data
data_bike.data = DataTools.merge_dataframes(
    data_bike.data, bike_station.data, "id", "id", ["bike_stands", "latitude", "longitude", "id_pollution"]
)

# Merge additional information (e.g., location, capacity) into bike or pollution data
data_pollution.data = DataTools.merge_dataframes(
    data_pollution.data, pollution_station.data, "id", "id", ["latitude", "longitude"]
)

del pollution_station
del bike_station

### 📊 Étape 5 : Calculer la capacité et les statistiques d'utilisation des vélos
Nous analysons l’utilisation des vélos par ville : combien de vélos sont disponibles, combien sont utilisés quotidiennement ou à différentes périodes de la journée.

In [ ]:
# Calculate the total number of bike slots available in the specified city
bike_count_amiens = DataTools.calul_capacity(data_bike.data, "amiens")
# Calculate the total number of bike slots available in the specified city
bike_count_marseille = DataTools.calul_capacity(data_bike.data, "marseille")

print(f"Number of slots in Amiens: {bike_count_amiens}")
print(f"Number of slots in Marseille: {bike_count_marseille}")

# Compute bike usage statistics such as daily and hourly usage
dailyuse_amiens, period_use_amiens, useperhour_amiens = DataTools.calculate_use(
    data_bike.data[data_bike.data["city"] == "amiens"]
)
# Compute bike usage statistics such as daily and hourly usage
dailyuse_marseille, period_use_marseille, useperhour_marseille = DataTools.calculate_use(
    data_bike.data[data_bike.data["city"] == "marseille"]
)

### 🔍 Étape 6 : Réaliser une analyse de corrélation
Nous examinons les liens entre l’utilisation des vélos et différents paramètres météorologiques (température, humidité...) ou environnementaux (polluants comme NO2, O3...).

In [ ]:
# Perform correlation analysis between bike usage and environmental factors
DataTools.corr_analysis(
    [dailyuse_amiens, data_weather.data],
    ["total_bikes_used", ["temp", "temp_max", "temp_min", "humidity", "speed", "clouds"]],
)

# Perform correlation analysis between bike usage and environmental factors
DataTools.corr_analysis(
    [dailyuse_amiens, data_pollution.data],
    ["total_bikes_used", ["NO", "NO2", "NOX as NO2", "O3", "PM10", "PM2.5"]],
)

# Perform correlation analysis between bike usage and environmental factors
DataTools.corr_analysis(
    [dailyuse_marseille, data_weather.data],
    ["total_bikes_used", ["temp", "temp_max", "temp_min", "humidity", "speed", "clouds"]],
)

# Perform correlation analysis between bike usage and environmental factors
DataTools.corr_analysis(
    [dailyuse_marseille, data_pollution.data],
    ["total_bikes_used", ["NO", "NO2", "NOX as NO2", "O3", "PM10", "PM2.5"]],
)

### 🤖 Étape 7 : Prédire l'utilisation des vélos
Dans cette étape, nous utilisons des modèles de régression et de classification pour prédire l'utilisation des vélos en fonction des données météorologiques et environnementales. Nous évaluons également les performances des modèles et effectuons des prédictions sur de nouvelles données. Les modèles peuvent être sauvegardés pour une utilisation future.

In [ ]:
results = DataTools.predict_bike_usage(
    usage_data=dailyuse_amiens, 
    weather_data=data_weather.data,
    pollution_data=data_pollution.data,
    threshold=100 # Optional: value above which usage is considered "high"
)
# After calling predict_bike_usage, add the following code to use the results:
# Check if the prediction was successful
if results and 'error' not in results:
    print("\n===== USING PREDICTION RESULTS =====")
    
    # 1. Access the regression model metrics
    if 'regression' in results:
        reg_metrics = results['regression']
        print(f"Regression model R² score: {reg_metrics['r2']:.4f}")
        print(f"RMSE: {reg_metrics['rmse']:.2f} bikes")
        
        # Make a prediction with the regression model for a new data point
        # Example: predict bike usage for a specific set of features
        new_data = pd.DataFrame({
            'day_of_week': [0],  # Monday
            'month': [6],        # June
            'is_weekend': [0],   # Not weekend
            'temp': [25],        # 25°C
            'humidity': [50]     # 50% humidity
            # Add other features as needed
        })
        
        # Make sure to use only the features that the model was trained on
        missing_cols = set(reg_metrics['features']) - set(new_data.columns)
        for col in missing_cols:
            new_data[col] = 0  # Fill missing columns with default values
            
        new_data = new_data[reg_metrics['features']]  # Reorder columns
        
        # Scale the data using the same scaler used for training
        scaled_data = reg_metrics['scaler'].transform(new_data)
        
        # Make prediction
        predicted_usage = reg_metrics['model'].predict(scaled_data)[0]
        print(f"Predicted bike usage for a 25°C Monday in June: {predicted_usage:.0f} bikes")
    
    # 2. Use the classification model for high/low usage prediction
    if 'classification' in results:
        print("\nBinary classification performance:")
        print(f"Accuracy: {results['classification']['accuracy']:.2f}")
        
        # You could predict if usage will be high or low for new data
        if reg_metrics and 'model' in results['classification']:
            high_usage = results['classification']['model'].predict(scaled_data)[0]
            print(f"High usage day? {'Yes' if high_usage else 'No'}")
    
    # 3. Use the category classification model
    if 'category_classification' in results:
        print("\nCategory classification performance:")
        print(f"Accuracy: {results['category_classification']['accuracy']:.2f}")
        
        # Predict usage category (low/medium/high) for new data
        if reg_metrics and 'model' in results['category_classification']:
            category = results['category_classification']['model'].predict(scaled_data)[0]
            print(f"Usage category: {category}")
    
    # 4. You could save the models for future use
# Importing necessary modules
    import pickle
    
    # Save regression model
    if 'regression' in results:
        with open('bike_usage_regression_model.pkl', 'wb') as f:
            pickle.dump({
                'model': results['regression']['model'],
                'scaler': results['regression']['scaler'],
                'features': results['regression']['features']
            }, f)
        print("\nRegression model saved to 'bike_usage_regression_model.pkl'")
        
    print("\n===== END OF RESULTS USAGE =====")
else:
    print("Prediction failed or returned no results.")

### 🚀 Etape 8 : Side Project

Nous avons avancé sur des parties du code qui peuvent être intégrées avec le reste pour une utilisation plus réelle et moins académique. Ces améliorations visent à rendre le projet plus pratique et applicable dans des contextes concrets.

Ici, nous appelons des fonctions qui nous permettent de tracer sur une carte toutes les stations de vélo avec le taux de remplissage à un temps donné. Les marqueurs affichent les stations avec une vue maps ou satellite. Si l'on passe la souris sur un marqueur, un popup s'ouvre indiquant le nombre de places, le nombre de vélos et le taux de remplissage.

In [ ]:
station_data = data_bike.data.drop_duplicates(subset=["id"])

# Create map with bike stations
DataTools.create_bike_station_map(
    station_data_list=[station_data],
    bike_data_list=[data_bike.data],
    specified_time=specified_time
)

Ici, nous faisons un appel à l'API publique de JCDecaux qui gère les vélos d'Amiens. Les données sont récupérées puis stockées dans un fichier Excel. Ces données sont mises à jour toutes les minutes, donc une boucle while nous permet de générer un fichier par minute.  
Ces données peuvent ensuite directement être utilisées par les autres fonctions, par exemple pour prédire l'utilisation des vélos durant l'heure suivante (chose non implémentée ici).

In [ ]:
print("Liste des contrats disponibles :")
DataTools.get_api_data(contracts_url, api_key)

# Récupérer les stations pour Marseille et Amiens
marseille_stations = DataTools.get_station_names(api_key, "Marseille")
amiens_stations = DataTools.get_station_names(api_key, "Amiens")
toulouse_stations = DataTools.get_station_names(api_key, "Toulouse")

while(True):
    start_time = time.time()
    # Sauvegarder les données dans un fichier CSV pour Marseille et Amiens
    now = datetime.now()
    date_str = now.strftime("%Y_%m_%d_%H_%M")  # Format : annee_mois_jour_heure_minute
    
    if marseille_stations:
        DataTools.save_station_data_to_xlsx(marseille_stations, "Marseille", api_key, date_str)

    # if toulouse_stations:
    #    DataTools.save_station_data_to_xlsx(toulouse_stations, "Toulouse", api_key, date_str)

    if amiens_stations:
        DataTools.save_station_data_to_xlsx(amiens_stations, "Amiens", api_key, date_str)
        
    time.sleep(max(0, 60 - (time.time() - start_time)))

### ✅ Étape finale : Nettoyage
Pour conclure, nous libérons explicitement la mémoire utilisée pour assurer de bonnes performances et éviter les fuites mémoire.

In [ ]:
# Explicitly free memory
gc.collect()
print("Program completed successfully. Memory freed.")